In [34]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-3-haiku-20240307"

In [35]:
# Helper functions
from anthropic.types import Message

def add_user_message(messages, message):
    user_message = {"role": "user", "content": message.content if isinstance(message, Message) else message}
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {"role": "assistant", "content": message.content if isinstance(message, Message) else message}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    if tools:
        params["tools"] = tools

    message = client.messages.create(**params)
    return message

def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )    

In [ ]:
## Tool definition

from anthropic.types import ToolParam
from datetime import datetime
def get_current_datetime(datetime_format="%Y-%m-%d %H:%M:%S"):
    if not datetime_format:
        raise ValueError("date_format cannot be empty")
    now = datetime.now()
    return now.strftime(datetime_format)

get_current_datetime_schema = ToolParam(
{
  "name": "get_current_datetime",
  "description": "Get the current date and time as a formatted string",
  "input_schema": {
    "type": "object",
    "properties": {
      "datetime_format": {
        "type": "string",
        "description": "A string specifying the format of the returned datetime. Uses Python's strftime format codes.",
        "default": "%Y-%m-%d %H:%M:%S"
      }
    },
    "required": []
  }
}
)

In [ ]:
## Example usage

messages = []

messages.append(
    {
        "role": "user",
        "content": "What is the exact time, formatted as 'HH:MM:SS'",
    }
)

response = client.messages.create(
    model=model,
    messages=messages,
    max_tokens=1000,
    tools=[get_current_datetime_schema],
)

messages.append({"role": "assistant", "content": response.content})
messages

[{'role': 'user',
  'content': "What is the exact time, formatted as 'HH:MM:SS'"},
 {'role': 'assistant',
  'content': [ToolUseBlock(id='toolu_01LPVrvJZBjDmjY69qmSqhVW', input={'datetime_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')]}]

In [44]:
# Example of running the tool

result = get_current_datetime(**response.content[0].input)

In [ ]:
## Append the tool result to the conversation

messages.append({
    "role": "user",
    "content":[
        {
            "type": "tool_result",
            "tool_use_id": response.content[0].id,
            "content": result,
            "is_error": False
        }
    ]
})
messages

[{'role': 'user',
  'content': "What is the exact time, formatted as 'HH:MM:SS'"},
 {'role': 'assistant',
  'content': [ToolUseBlock(id='toolu_01LPVrvJZBjDmjY69qmSqhVW', input={'datetime_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01LPVrvJZBjDmjY69qmSqhVW',
    'content': '19:05:22',
    'is_error': False}]}]

In [ ]:
# Final response from the model after tool execution
client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    tools=[get_current_datetime_schema]
)

Message(id='msg_014YRiSRhVZPia9cW5BtU7wX', content=[TextBlock(citations=None, text="The current time, formatted as 'HH:MM:SS', is 19:05:22.", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=466, output_tokens=27, server_tool_use=None, service_tier='standard', cache_creation={'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}))

In [ ]:
## Tool execution
import json

def run_tool(tool_name, tool_input):
    if tool_name == "get_current_datetime":
        return get_current_datetime(**tool_input)
    else:
        raise ValueError(f"Unknown tool: {tool_name}")

def run_tools(message):
    tool_requests = [block for block in message.content if block.type == "tool_use"]
    tool_result_blocks = []

    for tool_request in tool_requests:
        if tool_request.name == "get_current_datetime":
            try:
                tool_output = get_current_datetime(**tool_request.input)
                tool_result_block = {
                    "type": "tool_result",
                    "tool_use_id": tool_request.id,
                    "content": json.dumps(tool_output),
                "is_error": False
            }
            except Exception as e:
                tool_result_block = {
                    "type": "tool_result",
                    "tool_use_id": tool_request.id,
                    "content": str(e),
                    "is_error": True
                }

            tool_result_blocks.append(tool_result_block)

In [ ]:
## Run a conversation with tool use

def run_conversation(messages):
    while True:
        response = chat(messages, tools=[get_current_datetime_schema])
        add_assistant_message(messages, response)
        print(text_from_message(response))

        if response.stop_reason != "tool_use":
            break

        run_tools(response)